In [8]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')

driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - There is no [mac64] chromedriver for browser 87.0.4280 in cache
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/alementuev/.wdm/drivers/chromedriver/mac64/87.0.4280.88]
<ipython-input-8-dab0fdd945fb>:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)


In [17]:
url = 'https://www.linkedin.com/in/alementuev/'
driver.get(url)


In [18]:
page_source = driver.page_source

In [25]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

In [45]:
import re

cover_image = soup.find('figure', class_='cover-img onload lazy-loaded')
cover_image_url = re.search(r'url\("(.*?)"\)', cover_image['style'])[1]

In [46]:
profile_image = soup.find('img', class_='artdeco-entity-image artdeco-entity-image--profile artdeco-entity-image--circle-8 top-card-layout__entity-image top-card__profile-image top-card__profile-image--real-image onload lazy-loaded')
profile_image_url = profile_image['src']

In [28]:
title = soup.find('h1', class_='top-card-layout__title').text
title = title.strip()

In [51]:
headline = soup.find('h2', class_='top-card-layout__headline').text.strip()

In [31]:
location = soup.find('span', class_='top-card__subline-item').text

In [32]:
connections = soup.find('span', class_='top-card__subline-item top-card__subline-item--bullet').text

In [58]:
position_details = soup.find('div', {'data-section':'currentPositionsDetails'})
company = position_details.find('span', class_='top-card-link__description').text.strip()
company_image_url = position_details.find('img', class_='artdeco-entity-image artdeco-entity-image--company artdeco-entity-image--square-1 artdeco-entity-image--ghost artdeco-entity-image--ghost-company top-card-link__logo lazy-loaded')['src']


In [57]:
education_details = soup.find('div', {'data-section': 'educationsDetails'})
school_name = education_details.find('span', class_='top-card-link__description').text.strip()
school_url = education_details.find('a', class_='top-card-link top-card-link--link')['href']
school_image_url = education_details.find('img', class_='artdeco-entity-image artdeco-entity-image--school artdeco-entity-image--square-1 top-card-link__logo lazy-loaded')['src']

In [59]:
about = soup.find('section', class_='summary pp-section')
about_summary = about.find('p').text.strip()

In [77]:
class ExperienceItem:
    def __init__(self, position, start_date, end_date, location, description):
        self.position = position
        self.company = company
        self.start_date = start_date
        self.end_date = end_date
        self.description = description
        self.location = location

class ExperienceGroup:
    def __init__(self, company, items):
        self.company = company
        self.items = items


In [126]:
def parse_experience_card(node):
    classes = node['class']
    if 'experience-group' in classes:
        return parse_experience_group(node)
    if 'experience-item' in classes:
        return parse_experience_item(node)

    raise ValueError(f'Unexpected node: {node}')

def parse_experience_group(node):
    company = node.find('h4', class_='experience-group-header__company').text.strip()
    positions_list = node.find('ul', class_='experience-group__positions')
    cards = positions_list.findAll('li')
    items = list(map(parse_experience_group_position, cards))
    return ExperienceGroup(company, items)

def parse_experience_group_position(node):
    position = parse_experience_group_position_title(node)
    start, end = parse_date_range(node)
    location = node.find('p', class_='experience-group-position__location experience-group-position__meta-item')
    description = parse_experience_position_description(node)
    return ExperienceItem(position, start, end, location, description)

def parse_experience_item(node):
    company = node.find('h4', class_='result-card__subtitle').text.strip()
    position = parse_experience_group_position_title(node)
    start, end = parse_date_range(node)
    location = node.find('p', class_='experience-item__location experience-item__meta-item')
    description = parse_experience_position_description(node)
    item = ExperienceItem(position, start, end, location, description)
    return ExperienceGroup(company, [item])


def parse_experience_group_position_title(node):
    return node.find('h3', class_='result-card__title').text.strip()

def parse_experience_position_description(node):
    description = node.find('p', class_='show-more-less-text__text--more') or node.find('p', class_='show-more-less-text__text--less')
    return description.get_text('\n', strip=True) if description else None

def parse_date_range(node):
    date_range = node.find('span', class_='date-range')
    time_list = date_range.findAll('time')
    time_values = list(map(lambda time: time.text.strip(), time_list))
    if len(time_values) == 2:
        return time_values[0], time_values[1]
    if len(time_values) == 1:
        return time_values[0], None  # still working there
    raise ValueError(f'Unexpected time list: ${time_list}')

In [127]:
experience_list = soup.find('ul', class_='experience__list')
experience_cards = experience_list.findAll('li', recursive=False)

experience_groups = list(map(parse_experience_card, experience_cards))

In [128]:
for group in experience_groups:
    print(group.company)
    for item in group.items:
        print(f'\t{item.position}')
        print(f'\t\t{item.start_date}-{item.end_date or "Present"}')
        print(f'\t\t{item.description}')

Alex Saves The Day
	Software Engineering Consultant
		Sep 2020-Present
		None
Space Madness
	Director Of Pixels
		May 2015-Present
		Carefully crafted and published a whole bunch of delightful dev tools and interactive experiences for Unity.
Most notable projects:
- Lunar Mobile Console: a high-performance native iOS/Android console for testing and debugging. Integrated in over 10,000 Unity games (Niantic used it to debug Pokémon GO).
- Lunar Plugin for Unity: Quake-style terminal for Unity editor.
- Quake One In Unity: Quake One built from scratch in Unity.
- Super Unity Bros: pixel perfect Super Mario Bros remake
- Battle City: pixel perfect Battle City remake
Apptentive
	Mobile Tech Lead
		Mar 2018-Jan 2020
		Lead Apptentive SDK project development (responsible key technical decisions and architectural design). Worked closely with the product team to design "Feedback" and "Love Anything" mobile projects. Provided training and mentorship for company employees. Conducted technical int